In [1]:
import sys
sys.path.append("/mnt/e/WORK_DL/VisionLab/src")

#Import library
import os
import glob
import cv2
import dlib
import numpy as np
import matplotlib.pyplot as plt

from clearml import Task, Logger
import tqdm

from utils.count_ced_for_points import val_ced_calculate

task = Task.init(
    project_name='FaceKeypointDetection',
    task_name="dlib test",
    tags=["dlib", "test"]
)
logger = Logger.current_logger()

# Read an image
image = cv2.imread("/mnt/e/WORK_DL/datasets/landmarks_task/300W/test/261068_2.jpg")

paths = glob.glob("/mnt/e/WORK_DL/datasets/landmarks_task/Menpo/test/*.[jp][pn]g")

# Call face detector
detector = dlib.get_frontal_face_detector()

#Load the model of facial key points
predictor = dlib.shape_predictor("/mnt/e/WORK_DL/VisionLab/shape_predictor_68_face_landmarks.dat") # "shape_predictor_68_face_landmarks.dat"

gt_points = {}
predicted_points = {"dlib_test": {}}

for p in tqdm.tqdm(paths, ncols=150):
    markup = np.loadtxt(p.replace("png", "pts").replace("jpg", "pts"), comments=("version:", "n_points:", "{", "}"))
    flg = markup.shape[0] == 68
    if not flg:
        continue

    # Read an image
    image = cv2.imread(p)
    # Grayscale conversion
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    # Face Detection
    faces = detector(gray,1)
    # Loop through each face and draw the face rectangle and key points
    for i, face in enumerate(faces): #(x,y,w,h)


        # Draw a rectangular box
        # cv2.rectangle(image,(face.left(),face.top()),(face.right(),face.bottom()),(255,0,0),2)
        # Predict key points
        shape = predictor(image,face)
        # Get the coordinates of key points

        eyes_idx=[36,39,42,45]
        xmin, ymin, xmax, ymax = face.left(),face.top(),face.right(),face.bottom()
        flg2 = False
        if all(
            [markup[i][0] > xmin for i in eyes_idx] +
            [markup[i][1] > ymin for i in eyes_idx] +
            [markup[i][0] < xmax for i in eyes_idx] +
            [markup[i][1] < ymax for i in eyes_idx]
        ):
            basename = os.path.basename(p)
            predicted_points["dlib_test"][basename] = []

            markup = np.loadtxt(p.replace("png", "pts").replace("jpg", "pts"), comments=("version:", "n_points:", "{", "}"))
            gt_points[basename] = (markup[:, 0], markup[:, 1])
            # print('created')
            for pt in shape.parts():
                # Get the horizontal and vertical coordinates
                pt_position=[pt.x,pt.y]
                # Display/draw key points
                # cv2.circle(image,pt_position,3,(0,0,255),-1)
                predicted_points["dlib_test"][basename].append(pt_position)
            flg2 = True

            predicted_points["dlib_test"][basename] = np.asarray(predicted_points["dlib_test"][basename])
            # print( predicted_points["dlib_test"][basename].shape,len(shape.parts()), flg2)
            predicted_points["dlib_test"][basename] = (predicted_points["dlib_test"][basename][:, 0], predicted_points["dlib_test"][basename][:, 1])
            break

fig = val_ced_calculate(
    predicted_points=predicted_points,
    gt_points=gt_points,
    output_path="/mnt/e/WORK_DL/VisionLab/src/tasks_info/dlib_test",
    normalization_type="bbox",
    # left_eye_idx="36,39",
    # right_eye_idx="42,45",
    epoch=-1
)
logger.report_matplotlib_figure(
    title=f"Test dlib",
    series=f"CED@0.08",
    iteration=0,
    figure=fig,
    report_interactive=False,
)

task.close()

ClearML Task: overwriting (reusing) task id=bdec272de2e4490699a99a0378e4d0f8
2024-04-15 03:33:31,112 - clearml.Repository Detection - WARNING - Could not read Jupyter Notebook: No module named 'nbconvert'
2024-04-15 03:33:31,222 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/9670131bfe154f11b1f35546a11e4f69/experiments/bdec272de2e4490699a99a0378e4d0f8/output/log


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 898/898 [01:18<00:00, 11.50it/s]
